In [6]:
import pandas as pd

In [248]:
def prob_classA(total_distinct, train_set, column, distinct):
    info = []
    for i in range(1,len(column)-3):
        distinct_value = pd.value_counts(train_set[column[i]].values)
        start,j = 0,i-1
        arraycolumn = []
        while start<len(distinct[j]):
            get_idx = distinct[j][start]
            prob = distinct_value[get_idx]/total_distinct[0]
            arraycolumn.append(prob)
            start = start + 1
        info.append(arraycolumn)
    return info

def prob_classB(total_distinct, train_set, column, distinct):
    info = []
    for i in range(1,len(column)-3):
        distinct_value = pd.value_counts(train_set[column[i]].values)
        start,j = 0,i-1
        arraycolumn = []
        while start<len(distinct[j]):
            get_idx = distinct[j][start]
            prob = distinct_value[get_idx]/total_distinct[1]
            arraycolumn.append(prob)
            start = start + 1
        info.append(arraycolumn)
    return info  

In [249]:
def trainData(idxVal,df,idx,prev_idx):
    
    if idxVal == 1:
        train_set = df[idx:]
    else:
        if idxVal != 7:
            train_set = pd.concat([df[:prev_idx],df[idx:]])
        else:
            train_set = df[:idx] 
    return train_set

def testData(idxVal,df,idx,prev_idx):
    if idxVal == 1:
        test_set = df[:idx]
    else:
        if idxVal != 7:
            test_set = df[prev_idx:idx]
        else:
            test_set = df[idx:]
    return test_set

In [250]:
def is_found(distinct_train,idx,k):
    set_DistinctIdx = distinct_train[idx]
    i,index = 0,0
    found = True
    while i < len(set_DistinctIdx) and found:
        if set_DistinctIdx[i] == k:
            index = set_DistinctIdx[i]
            found = False
        else:   
            i = i + 1
    if index == k:
        return True
    else:
        return False

def search_prob(distinct_train,idx,k):
    set_DistinctIdx = distinct_train[idx]
    i = 0
    found = True
    while i < len(set_DistinctIdx) and found:
        if set_DistinctIdx[i] != k:
            i = i + 1
        else:   
            found = False
    return i

def calculate_TestS(test_set,distinct_train,class_A,column):
    info= []
    for idx in range(1,len(column)-3):
        d = test_set.iloc[:,idx]
        prob_Arr= []
        i = idx-1
        for k,value in enumerate(d):
            if is_found(distinct_train,i,value):
                prob = search_prob(distinct_train,i,value)
                tmp = class_A[i][prob]
            else:
                tmp = 0
            prob_Arr.append(tmp)
        info.append(prob_Arr)
    return info

    def calculate_TestS(test_set,distinct_train,class_B,column):
        info= []
        for idx in range(1,len(column)-3):
            d = test_set.iloc[:,idx]
            prob_Arr= []
            i = idx-1
            for k,value in enumerate(d):
                if is_found(distinct_train,i,value):
                    prob = search_prob(distinct_train,i,value)
                    tmp = class_B[i][prob]
                else:
                    tmp = 0
                prob_Arr.append(tmp)
            info.append(prob_Arr)
        return info

In [251]:
def get_class(a,b,distinct,total_class,test_set,idx):
    precision, Recall, TN ,TP, FN, FP, d = 0,0,0,0,0,0,0
    idx_v = idx - len(test_set)
    for i in range(len(a)):
        if a[i]>b[i]:
            d = test_set.iloc[:,10]
            if d[idx_v]==(distinct[9][0]):
                TN = TN + 1
            else:
                FN = FN + 1
        else:
            d = test_set.iloc[:,10]
            if d[idx_v]==(distinct[9][1]):
                TP = TP + 1
            else:
                FP = FP + 1
        idx_v = idx_v + 1
    print("True Positive :",TP," False Positive: ",FP," True Negative: ",TN," False Negative: ",FN)
    accuracy = (TP/len(a) * 100)
    if (TP+FP) == 0 :
        precision = 0
    elif (TP+FN)==0:
        Recall = 0
    else:
        precision = (TP/(TP+FP)*100)
        Recall = (TP/(TP+FN)*100)
    print("precision: ",round(precision,2),"% Recall: ",round(Recall,2),"%")
    print("The accuracy is ", round(accuracy,2),'%')

In [252]:
def main():
 
    df = pd.read_csv('csv_result-labor1.csv')
    prev_idx = 0
    testset = int(len(df)/5)
    for i in range(1,6):
        idx = testset * i
        train_set = trainData(i,df,idx,prev_idx)
        test_set = testData(i,df,idx,prev_idx)

        print("============== ",i," FOLD ===================")

        column = [col for col in df.columns]

        distinct_train = [train_set[column[idx]].unique() for idx in range(1,len(column))]
        distinct_test = [test_set[column[idx]].unique() for idx in range(1,len(column))]

        total_distinct = pd.value_counts(train_set['class'].values)
        total_class = pd.value_counts(test_set['class'].values)

        prob_disA = total_distinct[0]/len(train_set)
        prob_disB = total_distinct[1]/len(train_set)

        class_A = prob_classA(total_distinct,train_set,column,distinct_train)
        class_B = prob_classB(total_distinct,train_set,column,distinct_train)

        resultA = calculate_TestS(test_set,distinct_train,class_A,column)
        resultB = calculate_TestS(test_set,distinct_train,class_B,column)

        a = mul_classA(test_set,resultA,prob_disA)
        b = mul_classB(test_set,resultB,prob_disB)

        get_class(a,b,distinct_train,total_class,test_set,idx)
        prev_idx = idx


In [253]:
main()

==============  1  FOLD ===================
True Positive : 11  False Positive:  0  True Negative:  0  False Negative:  0
precision:  100.0 % Recall:  100.0 %
The accuracy is  100.0 %
==============  2  FOLD ===================
True Positive : 4  False Positive:  7  True Negative:  0  False Negative:  0
precision:  36.36 % Recall:  100.0 %
The accuracy is  36.36 %
==============  3  FOLD ===================
True Positive : 6  False Positive:  5  True Negative:  0  False Negative:  0
precision:  54.55 % Recall:  100.0 %
The accuracy is  54.55 %
==============  4  FOLD ===================
True Positive : 10  False Positive:  1  True Negative:  0  False Negative:  0
precision:  90.91 % Recall:  100.0 %
The accuracy is  90.91 %
==============  5  FOLD ===================
True Positive : 0  False Positive:  11  True Negative:  0  False Negative:  0
precision:  0 % Recall:  0 %
The accuracy is  0.0 %
